In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","24G") \
        .getOrCreate()

spark

In [2]:
import pandas as pd

replication_df = spark.createDataFrame(pd.DataFrame(list(range(1,1000)),columns=['replication_id'])).repartition(1000, 'replication_id')

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


outSchema = StructType([StructField('replication_id', IntegerType(), True),
            StructField('sil_score', DoubleType(), True),
            StructField('num_clusters', IntegerType(), True),
            StructField('min_samples', IntegerType(), True),
            StructField('min_cluster_size', IntegerType(), True)])


@pandas_udf(outSchema, PandasUDFType.GROUPED_MAP)
def run_model(df_pandas):


    # Return result as a pandas data frame
    result = pd.DataFrame({'replication_id': replication_id, 'sil_score': 2,
                           'num_clusters': 3, 'min_samples': 4,
                           'min_cluster_size': 5}, index=[0])

    return result



results = replication_df.groupby("replication_id").apply(run_model)

/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


In [4]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",True)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",False)

results.count()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 255, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 88, in dump_stream
    for batch in iterator:
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py", line 248, in init_stream_yield_batches
   

In [ ]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",False)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",False)

results.count()

In [ ]:
spark.conf.set("spark.sql.pyspark.jvmStacktrace.enabled",False)
spark.conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled",True)

results.count()